Prediction model for Rel. substance degradation based on available analytic solutions to chemical kinetics. <br>
Linear:
\begin{align}
\frac{d[B]}{dt} =  k
\end{align}
Pseudo 0.5 order up to third order
\begin{align}
\frac{d[B]}{dt} = k [A] ^{\frac{1}{2}}, \frac{d[B]}{dt} = k [A], \frac{d[B]}{dt} = k [A] ^{\frac{3}{2}}
\end{align}

\begin{align}
\frac{d[B]}{dt} = k [A] ^{2}, \frac{d[B]}{dt} = k [A] ^{3}, 
\end{align}

Exponential model
\begin{align}
\frac{d[B]}{dt} = k \frac{[B]}{t}
\end{align}

Gustafsson-Holden model
\begin{align}
\frac{d[B]}{dt} = \frac{k_1}{k_2} [B] \left( \frac{t}{k_2} +1 \right) ^{-1}
\end{align}

Sequential models:  
First order - zero order:
\begin{align}
\frac{d[B]}{dt} = k_1 [A] - k_2
\end{align}

Zero order - first order:
\begin{align}
\frac{d[B]}{dt} = k_1  - k_2[B]
\end{align}

First order - first order:
\begin{align}
\frac{d[B]}{dt} = k_1[A]  - k_2[B]
\end{align}

In [ ]:
#!pip install numpy
#!pip install matplotlib
#!pip install pandas
#!pip install scipy
#!pip install IPython
#!pip install ipywidgets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy.optimize import curve_fit
from IPython.display import Math, display
from scipy.stats import t
from ipywidgets import FileUpload, interact, IntSlider
import io
import csv
from matplotlib.ticker import StrMethodFormatter
import os

In [ ]:
from tkinter import filedialog
from tkinter import Tk

# Create a Tkinter root window (it won't be shown)
root = Tk()
root.withdraw()

# Open a file dialog for the user to select a CSV file
file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])

# Check if a file was selected
if file_path:
    # Read the CSV file using pandas
    data = pd.read_csv(file_path)
    print(data.head())

    # Check if the DataFrame is empty
    if not data.empty:
        product_name = input("Product name: ")

        # DataFrame operations
        headers = data.columns.tolist()
        RelSub = pd.DataFrame(data, columns=headers, dtype="float").values
    else:
        print("DataFrame is empty.")
else:
    print("No file selected.")

In [ ]:
#RelSub_df = pd.DataFrame(data, columns=headers, dtype="float")
#RelSub = pd.DataFrame(data, columns=headers, dtype="float").values
#RelSub_df.head()

if data:
    #RelSub_df = pd.DataFrame(data, columns=headers, dtype="float")
    RelSub = pd.DataFrame(data, columns=headers, dtype="float").values
    #RelSub_df.head()
else:
    print("File upload failed or took too long.")

In [ ]:
#Specification limit for the related substance
spec = float(input("Specification limit: "))
print(f'Specification limit for {product_name} = {spec:.2f} %')

In [ ]:
x_data = RelSub[:,0]
y_data = RelSub[:,1]

In [ ]:
print(x_data)
print(y_data)

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)")
plt.ylabel("Related substance (%)")
plt.show()

Choosing the initial conditions for quations with single constant k

In [ ]:
initial=[1e-10, 0.01]

Choosing initial conditions for two constant k models

In [ ]:
initial_TC = [1e-10,0.01,0.1]

All the functions as analytical solutions to the differential equations written at the beginning

In [ ]:
def zero_f(t, c0, k_zero):
    return c0 + k_zero*t

def half_f(t, c0, k_half):
    return c0 + (100 - (np.sqrt(100)-(k_half*t*0.5))**2)

def first_f(t, c0, k_first):
    return c0 + 100 - 100*np.exp(-k_first*t)

def first_half_f(t, c0, k_firsth):
    return c0 + 100 - (1/((k_firsth*t*0.5)+(1/np.sqrt(100))))**2

def second_f(t, c0, k_second):
    return c0 + 100 - 1/((1/100)+(k_second*t))

def third_f(t, c0, k_third):
    return c0 + 100 - 1/(np.sqrt((1/(100**2))+(2*abs(k_third)*t)))

def exp_f(t, c0, k_exp):
    return c0 + 100 - ((100*t)**k_exp)

def guho_f(t, c0, k1_gh, k2_gh):
    return c0*(((t/k2_gh)+1)**k1_gh)

def seq_first_z_f(t, c0, k1_fz, k2_fz):
    return c0 + 100 - 100*np.exp(-k1_fz*t) - k2_fz*t

def seq_zero_f_f(t, c0, k1_zf, k2_zf):
    return k1_zf*t + c0*np.exp(-k2_zf*t)

def seq_first_f_f(t, c0, k1_ff, k2_ff):
    return c0 + ((k1_ff*100)/(k2_ff-k1_ff))*(np.exp(-k1_ff*t) - np.exp(-k2_ff*t))

def seq_pseudo_f(t, c0, k1_p, k2_p):
    return c0 + ((k1_p**k2_p)*(t*k2_p))/((scipy.special.factorial(k2_p))*np.exp(k1_p*t))

Non linear curve fitting for each model separately

In [ ]:
popt_z, pcov_z = curve_fit(zero_f, x_data, y_data, p0=initial, bounds=((1e-10,1e-10),(np.inf, np.inf)))
popt_h, pcov_h = curve_fit(half_f, x_data, y_data, p0=initial, bounds=((1e-10,1e-10),(np.inf, np.inf)))
popt_f, pcov_f = curve_fit(first_f, x_data, y_data, p0=initial, bounds=((1e-10,1e-10),(np.inf, np.inf)))
popt_fh, pcov_fh = curve_fit(first_half_f, x_data, y_data, p0=initial, bounds=((1e-10,1e-10),(np.inf, np.inf)))
popt_s, pcov_s = curve_fit(second_f, x_data, y_data, p0=initial, bounds=((1e-10,1e-10),(np.inf, np.inf)))
popt_t, pcov_t = curve_fit(third_f, x_data, y_data, p0=initial, bounds=((1e-10,1e-10),(np.inf, np.inf)))
popt_exp, pcov_exp = curve_fit(exp_f, x_data, y_data, p0=initial, bounds=((1e-10,1e-10),(np.inf, np.inf)))
popt_gh, pcov_gh = curve_fit(guho_f, x_data, y_data, p0=initial_TC, bounds=((1e-10,1e-10,1e-10),(np.inf, np.inf, np.inf)))
popt_fz, pcov_fz = curve_fit(seq_first_z_f, x_data, y_data, p0=initial_TC, bounds=((1e-10,1e-10,1e-10),(np.inf, np.inf, np.inf)))
popt_zf, pcov_fz = curve_fit(seq_zero_f_f, x_data, y_data, p0=initial_TC, bounds=((1e-10,1e-10,1e-10),(np.inf, np.inf, np.inf)))
popt_ff, pcov_ff = curve_fit(seq_first_f_f, x_data, y_data, p0=initial_TC, bounds=((1e-10,1e-10,1e-10),(np.inf, np.inf, np.inf)))
popt_ps, pcov_ps = curve_fit(seq_pseudo_f, x_data, y_data, p0=initial_TC, bounds=((1e-10,1e-10,1e-10),(np.inf, np.inf, np.inf)))

Extraction of the fitted parameters

In [ ]:
x_model = np.linspace(min(x_data), max(x_data), num=100)

c0_opt_z, k_opt_z = popt_z
y_model_z = zero_f(x_model, c0_opt_z, k_opt_z)

c0_opt_h, k_opt_h = popt_h
y_model_h = half_f(x_model, c0_opt_h, k_opt_h)

c0_opt_f, k_opt_f = popt_f
y_model_f = first_f(x_model, c0_opt_f, k_opt_f)

c0_opt_fh, k_opt_fh = popt_fh
y_model_fh = first_half_f(x_model, c0_opt_fh, k_opt_fh)

c0_opt_s, k_opt_s = popt_s
y_model_s = second_f(x_model, c0_opt_s, k_opt_s)

c0_opt_t, k_opt_t = popt_t
y_model_t = third_f(x_model, c0_opt_t, k_opt_t)

c0_opt_exp, k_opt_exp = popt_exp
y_model_exp = exp_f(x_model, c0_opt_exp, k_opt_exp)

c0_opt_gh, k1_opt_gh, k2_opt_gh = popt_gh
y_model_gh = guho_f(x_model, c0_opt_gh, k1_opt_gh, k2_opt_gh)

c0_opt_fz, k1_opt_fz, k2_opt_fz = popt_fz
y_model_fz = seq_first_z_f(x_model, c0_opt_fz, k1_opt_fz, k2_opt_fz)

c0_opt_zf, k1_opt_zf, k2_opt_zf = popt_zf
y_model_zf = seq_zero_f_f(x_model, c0_opt_zf, k1_opt_zf, k2_opt_zf)

c0_opt_ff, k1_opt_ff, k2_opt_ff = popt_ff
y_model_ff = seq_first_f_f(x_model, c0_opt_ff, k1_opt_ff, k2_opt_ff)

c0_opt_ps, k1_opt_ps, k2_opt_ps = popt_ps
y_model_ps = seq_pseudo_f(x_model, c0_opt_ps, k1_opt_ps, k2_opt_ps)

print(f'Zero order parameters ({c0_opt_z:.4f}, {k_opt_z:.4f})')
print(f'Half order parameters ({c0_opt_h:.4f}, {k_opt_h:.4f})')
print(f'First order parameters ({c0_opt_f:.4f}, {k_opt_f:.4f})')
print(f'First/half order parameters ({c0_opt_fh:.4f}, {k_opt_fh:.4f})')
print(f'Second order parameters ({c0_opt_s:.4f}, {k_opt_s:.4f})')
print(f'Third order parameters ({c0_opt_t:.4f}, {k_opt_t:.4f})')
print(f'Exponential parameters ({c0_opt_exp:.4f}, {k_opt_exp:.4f})')
print(f'Gustafson Holden parameters ({c0_opt_gh:.4f}, {k1_opt_gh:.4f}, {k2_opt_gh:.4f})')
print(f'Seq. first/zero parameters ({c0_opt_fz:.4f}, {k1_opt_fz:.4f}, {k2_opt_fz:.4f})')
print(f'Seq. zero/zero parameters ({c0_opt_zf:.4f}, {k1_opt_zf:.4f}, {k2_opt_zf:.4f})')
print(f'Seq. first/first parameters ({c0_opt_ff:.4f}, {k1_opt_ff:.4f}, {k2_opt_ff:.4f})')
print(f'Pseudo exponential order parameters ({c0_opt_ps:.4f}, {k1_opt_ps:.4f}, {k2_opt_ps:.4f})')

In [ ]:
# Interpolate the model curve to the same points as the data
y_model_interp_z = np.interp(x_data, x_model, y_model_z)
y_model_interp_h = np.interp(x_data, x_model, y_model_h)
y_model_interp_f = np.interp(x_data, x_model, y_model_f)
y_model_interp_fh = np.interp(x_data, x_model, y_model_fh)
y_model_interp_s = np.interp(x_data, x_model, y_model_s)
y_model_interp_t = np.interp(x_data, x_model, y_model_t)
y_model_interp_exp = np.interp(x_data, x_model, y_model_exp)
y_model_interp_gh = np.interp(x_data, x_model, y_model_gh)
y_model_interp_fz = np.interp(x_data, x_model, y_model_fz)
y_model_interp_zf = np.interp(x_data, x_model, y_model_zf)
y_model_interp_ff = np.interp(x_data, x_model, y_model_ff)
y_model_interp_ps = np.interp(x_data, x_model, y_model_ps)

# Calculate the residuals
residuals_z = y_data - y_model_interp_z  # residuals = observed - fitted
residuals_h = y_data - y_model_interp_h
residuals_f = y_data - y_model_interp_f
residuals_fh = y_data - y_model_interp_fh
residuals_s = y_data - y_model_interp_s
residuals_t = y_data - y_model_interp_t
residuals_exp = y_data - y_model_interp_exp
residuals_gh = y_data - y_model_interp_gh
residuals_fz = y_data - y_model_interp_fz
residuals_zf = y_data - y_model_interp_zf
residuals_ff = y_data - y_model_interp_ff
residuals_ps = y_data - y_model_interp_ps

In [ ]:
model_list=["zero order","half order","first order","first half order","second order","third order",  "Exponential model", 
            "Gustafsson Holden", "Sequential first-zero order", "Sequential zero-first order", "Sequential first-first order",
           "Pseudo sequential model"]

In [ ]:
#Calculate the squared residuals for model comparison and choose the opimal model as a function
SqR_z = np.sum(np.square(residuals_z))
SqR_h = np.sum(np.square(residuals_h))
SqR_f = np.sum(np.square(residuals_f))
SqR_fh = np.sum(np.square(residuals_fh))
SqR_s = np.sum(np.square(residuals_s))
SqR_t = np.sum(np.square(residuals_t))
SqR_exp = np.sum(np.square(residuals_exp))
SqR_gh = np.sum(np.square(residuals_gh))
SqR_fz = np.sum(np.square(residuals_fz))
SqR_zf = np.sum(np.square(residuals_zf))
SqR_ff = np.sum(np.square(residuals_ff))
SqR_ps = np.sum(np.square(residuals_ps))

#Create an array of all sum of square residuals
"""
SQR = [SqR_z, SqR_h, SqR_f, SqR_fh, SqR_s, SqR_t, SqR_exp, SqR_gh, SqR_fz, SqR_zf, SqR_ff, SqR_ps]
"""
SQR = [SqR_z, SqR_h, SqR_f*0.5, SqR_fh, SqR_s, SqR_t, SqR_exp, 
       SqR_gh*1.05, SqR_fz*1.05, SqR_zf*1.05, SqR_ff*1.05, SqR_ps*1.05]

#Determine the lowest Sum of square residuals
min_index = SQR.index(min(SQR))

# Remove the first smallest value from the list
SQR_wo_min = SQR[:min_index] + SQR[min_index + 1:]

# Find the index of the second smallest value and record the value
second_min_index, SQR_2nd = min(enumerate(SQR_wo_min), key=lambda x: x[1])
#SQR_2nd=SQR_wo_min[second_min_index]

#Find the index of the second smallest value in the original SQR array
min_index_2nd = SQR.index(SQR_2nd)

In [ ]:
model_dict=dict(zip(model_list, SQR))
for index, (key, value) in enumerate(model_dict.items(), start=0):
    print(f'{index:<5} {key:<10} - {value:.4f}')
    
print('\n')    
print(f'First model choice, no. {min_index}, {model_list[min_index]}, {SQR[min_index]:.4f}')
print(f'Second model choice, no. {min_index_2nd}, {model_list[min_index_2nd]}, {SQR[min_index_2nd]:.4f}')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_z, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Zero order model', fontsize=15)
# Set the y-axis tick labels to display values with 2 decimal places
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel. Sub]_0+kt')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_h, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Pseudo half order model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)

plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Re.Sub.]_0 + 100 - \left(\sqrt{100} - \frac{kt}{2} \right)^2')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_f, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('First order model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - 100 \: e^ {\large{-kt}}')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_fh, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Pseudo first and half order model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - \left( \frac{1}{{\large \frac{kt}{2}} + {\large \frac{1}{\sqrt{100}}}} \right)^2')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_s, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Second order model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - \left( \frac{1}{kt + {\large \frac{1}{100}}} \right)')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_t, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Third order model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - \frac{1}{\sqrt{2kt + {\large \frac{1}{100^2}}}}')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_exp, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Exponential model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
# Set y-axis limits to show from 0 to spec
plt.ylim(min(y_data)-0.02, spec + 0.02)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - 100 \: t^ {k}')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_gh, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Gustafsson Holden model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 \left( \frac{t}{k_2} +1 \right) ^{k_1}')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_fz, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Sequential first-zero order', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 \left( 1 - e^ {\large{-k_1t}} \right) -k_2t')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_zf, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Sequential zero-first order', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = k_1t + [Rel.Sub.]_0 \:e^ {\large{-k_2t}}')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_ff, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Sequential first-first order', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + \frac{100k_1}{k_2-k_1} \left( e^ {\large{-k_1t}}-e^ {\large{-k_2t}}\right)')

In [ ]:
plt.scatter(x_data,y_data)
plt.xlabel("months (-)", fontsize=15)
plt.ylabel("Rel. Sub. (%)", fontsize=15)
plt.plot(x_model,y_model_ps, color='r')
plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.title('Pseudo sequential model', fontsize=15)
x_axis=np.arange(0, x_model[-1]+1,4)
plt.xticks(x_axis)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
plt.show()

Math(r'[Rel.Sub.] = [Rel.Sub.]_0 + \frac{k_1^{k_2} \: t^{k_2}}{k_2! \: e^{\large{k_1t}}}')

In [ ]:
print('The best model for the data is the **', model_list[min_index], '** model')
print('The 2nd best model for the data is the **', model_list[min_index_2nd], '** model')

#Choose the optimal function and the appropriate fitting parameters
SQR_functions = [zero_f, half_f, first_f, first_half_f, second_f, third_f, exp_f, guho_f, seq_first_z_f,
                seq_zero_f_f, seq_first_f_f, seq_pseudo_f]
optimal_model = SQR_functions[min_index]
optimal_model_2nd=SQR_functions[min_index_2nd]

fit_param = [popt_z, popt_h, popt_f, popt_fh, popt_s, popt_t, popt_exp, popt_gh, popt_zf, popt_fz, popt_ff, popt_ps]

best_fit_param = fit_param[min_index]
best_fit_param_2nd = fit_param[min_index_2nd]

y_optimal_model = optimal_model(x_model, *best_fit_param)
y_optimal_model_2nd = optimal_model_2nd(x_model, *best_fit_param_2nd)

#print(best_fit_param_2nd)
#print(y_optimal_model_2nd)

latex_strings = []
latex_strings.append(r'[Rel.Sub.] = [Rel. Sub]_0+kt')
latex_strings.append(r'[Rel.Sub.] = [Re.Sub.]_0 + 100 - \left(\sqrt{100} - \frac{kt}{2} \right)^2')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - 100 \: e^ {\large{-kt}}')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - \left( \frac{1}{{\large \frac{kt}{2}} + {\large \frac{1}{\sqrt{100}}}} \right)^2')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - \left( \frac{1}{kt + {\large \frac{1}{100}}} \right)')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - \frac{1}{\sqrt{2kt + {\large \frac{1}{100^2}}}}')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 - 100 \: t^ {k}')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 \left( \frac{t}{k_2} +1 \right) ^{k_1}')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + 100 \left( 1 - e^ {\large{-k_1t}} \right) -k_2t')
latex_strings.append(r'[Rel.Sub.] = k_1t + [Rel.Sub.]_0 \:e^ {\large{-k_2t}}')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + \frac{100k_1}{k_2-k_1} \left( e^ {\large{-k_1t}}-e^ {\large{-k_2t}}\right)')
latex_strings.append(r'[Rel.Sub.] = [Rel.Sub.]_0 + \frac{k_1^{k_2} \: t^{k_2}}{k_2! \: e^{\large{k_1t}}}')

Bootstrap method to calculate confidence intervals for each fitting parameter for the best model

In [ ]:
from scipy.optimize import minimize

if min_index < 7:
    # Number of bootstrap iterations
    n_iter = 1000

    # Store the bootstrap estimates for each parameter
    c0_bs = np.zeros(n_iter)
    k_bs = np.zeros(n_iter)


    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample = x_data[idx]
        y_resample = y_data[idx]
        # Fit the model to the resampled data
        bounds = [(1e-10, np.inf), (1e-10, np.inf)]
        res = minimize(lambda p: ((optimal_model(x_resample, p[0], p[1]) - y_resample)**2).sum(), best_fit_param, bounds=bounds, method='Nelder-Mead')
        c0_bs[i], k_bs[i] = res.x

    # Calculate the standard error of the bootstrap estimates
    c0_se = np.std(c0_bs, ddof=1)
    k_se = np.std(k_bs, ddof=1)

    # Calculate the t-value for the given confidence level and degree of freedom
    alpha = 0.05 # Confidence level
    df = len(x_data) - len(best_fit_param) # Degree of freedom
    tval = t.ppf(1 - alpha/2, df)

    # Calculate the confidence intervals for each parameter
    c0_ci = (best_fit_param[0] - tval * c0_se, best_fit_param[0] + tval * c0_se)
    k_ci = (best_fit_param[1] - tval * k_se, best_fit_param[1] + tval * k_se)


    # Print the confidence intervals
    print("Fitting parameters for",model_list[min_index],", the best model for the data")
    print(f"c0: {best_fit_param[0]:.2f}")
    print(f"k: {best_fit_param[1]:.4f}")
    print(f"Confidence intervals for c0: ({c0_ci[0]:.2f},{c0_ci[1]:.4f})")
    print(f"Confidence intervals for k: ({k_ci[0]:.2f},{k_ci[1]:.4f})")

else:
    n_iter = 1000

    # Store the bootstrap estimates for each parameter
    c0_bs = np.zeros(n_iter)
    k1_bs = np.zeros(n_iter)
    k2_bs = np.zeros(n_iter)

    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample = x_data[idx]
        y_resample = y_data[idx]
        # Fit the model to the resampled data
        bounds = [(1e-10, np.inf), (1e-10, np.inf), (1e-10, np.inf)]
        res = minimize(lambda p: ((optimal_model(x_resample, p[0], p[1], p[2]) - y_resample)**2).sum(), best_fit_param, bounds=bounds, method='Nelder-Mead')
        c0_bs[i], k1_bs[i], k2_bs[i] = res.x

    # Calculate the standard error of the bootstrap estimates
    c0_se = np.std(c0_bs, ddof=1)
    k1_se = np.std(k1_bs, ddof=1)
    k2_se = np.std(k2_bs, ddof=1)

    # Calculate the t-value for the given confidence level and degree of freedom
    alpha = 0.05 # Confidence level
    df = len(x_data) - len(best_fit_param) # Degree of freedom
    tval = t.ppf(1 - alpha/2, df)

    # Calculate the confidence intervals for each parameter
    c0_ci = (best_fit_param[0] - tval * c0_se, best_fit_param[0] + tval * c0_se)
    k1_ci = (best_fit_param[1] - tval * k1_se, best_fit_param[1] + tval * k1_se)
    k2_ci = (best_fit_param[2] - tval * k2_se, best_fit_param[2] + tval * k2_se)

    # Print the confidence intervals
    print("Fitting parameters for",model_list[min_index],", the best model for the data")
    print(f"c0: {best_fit_param[0]:.2f}")
    print(f"k1: {best_fit_param[1]:.4f}")
    print(f"k2: {best_fit_param[2]:.4f}")
    print(f"Confidence intervals for c0: ({c0_ci[0]:.2f},{c0_ci[1]:.4f})")
    print(f"Confidence intervals for k1: ({k1_ci[0]:.2f},{k1_ci[1]:.4f})")
    print(f"Confidence intervals for k2: ({k2_ci[0]:.2f},{k2_ci[1]:.4f})")

Bootstrap method to calculate confidence intervals for each fitting parameter for the 2nd best model

In [ ]:
from scipy.optimize import minimize

if min_index_2nd < 7:
    # Number of bootstrap iterations
    n_iter = 1000

    # Store the bootstrap estimates for each parameter
    c0_bs = np.zeros(n_iter)
    k_bs = np.zeros(n_iter)


    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample = x_data[idx]
        y_resample = y_data[idx]
        # Fit the model to the resampled data
        bounds = [(1e-10, np.inf), (1e-10, np.inf)]
        res = minimize(lambda p: ((optimal_model_2nd(x_resample, p[0], p[1]) - y_resample)**2).sum(), best_fit_param_2nd, bounds=bounds, method='Nelder-Mead')
        c0_bs[i], k_bs[i] = res.x

    # Calculate the standard error of the bootstrap estimates
    c0_se = np.std(c0_bs, ddof=1)
    k_se = np.std(k_bs, ddof=1)

    # Calculate the t-value for the given confidence level and degree of freedom
    alpha = 0.05 # Confidence level
    df = len(x_data) - len(best_fit_param_2nd) # Degree of freedom
    tval = t.ppf(1 - alpha/2, df)

    # Calculate the confidence intervals for each parameter
    c0_ci_2nd = (best_fit_param_2nd[0] - tval * c0_se, best_fit_param_2nd[0] + tval * c0_se)
    k_ci_2nd = (best_fit_param_2nd[1] - tval * k_se, best_fit_param_2nd[1] + tval * k_se)


    # Print the confidence intervals
    print("Fitting parameters for",model_list[min_index_2nd],", the second best model for the data")
    print(f"c0: {best_fit_param_2nd[0]:.2f}")
    print(f"k: {best_fit_param_2nd[1]:.4f}")
    print(f"Confidence intervals for c0: ({c0_ci_2nd[0]:.2f},{c0_ci_2nd[1]:.2f})")
    print(f"Confidence intervals for k: ({k_ci_2nd[0]:.4f},{k_ci_2nd[1]:.4f})")

else:
    n_iter = 1000

    # Store the bootstrap estimates for each parameter
    c0_bs = np.zeros(n_iter)
    k1_bs = np.zeros(n_iter)
    k2_bs = np.zeros(n_iter)

    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample = x_data[idx]
        y_resample = y_data[idx]
        # Fit the model to the resampled data
        bounds = [(1e-10, np.inf), (1e-10, np.inf), (1e-10, np.inf)]
        res = minimize(lambda p: ((optimal_model_2nd(x_resample, p[0], p[1], p[2]) - y_resample)**2).sum(), best_fit_param_2nd, bounds=bounds,  method='Nelder-Mead')
        c0_bs[i], k1_bs[i], k2_bs[i] = res.x

    # Calculate the standard error of the bootstrap estimates
    c0_se = np.std(c0_bs, ddof=1)
    k1_se = np.std(k1_bs, ddof=1)
    k2_se = np.std(k2_bs, ddof=1)

    # Calculate the t-value for the given confidence level and degree of freedom
    alpha = 0.05 # Confidence level
    df = len(x_data) - len(best_fit_param_2nd) # Degree of freedom
    tval = t.ppf(1 - alpha/2, df)

    # Calculate the confidence intervals for each parameter
    c0_ci_2nd = (best_fit_param_2nd[0] - tval * c0_se, best_fit_param_2nd[0] + tval * c0_se)
    k1_ci_2nd = (best_fit_param_2nd[1] - tval * k1_se, best_fit_param_2nd[1] + tval * k1_se)
    k2_ci_2nd = (best_fit_param_2nd[2] - tval * k2_se, best_fit_param_2nd[2] + tval * k2_se)

    # Print the confidence intervals
    print("Fitting parameters for",model_list[min_index_2nd],", the second best model for the data")
    print(f"c0: {best_fit_param_2nd[0]:.2f}")
    print(f"k1: {best_fit_param_2nd[1]:.4f}")
    print(f"k2: {best_fit_param_2nd[2]:.4f}")
    print(f"Confidence intervals for c0: ({c0_ci_2nd[0]:.2f},{c0_ci_2nd[1]:.2f})")
    print(f"Confidence intervals for k1: ({k1_ci_2nd[0]:.4f},{k1_ci_2nd[1]:.4f})")
    print(f"Confidence intervals for k2: ({k2_ci_2nd[0]:.4f},{k2_ci_2nd[1]:.4f})")

Bootstrap method for the calculation of confidence and prediction curves for the fitted curve of the best model

In [ ]:
n_points = 100

if min_index < 7:
    # Store the y-values for the model curve for each bootstrap iteration for confidence and prediction intervals
    y_model_bs = np.zeros((n_iter, n_points))
    y_pred_bs = np.zeros((n_iter, n_points))

    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample = x_data[idx]
        y_resample = y_data[idx]
        # Fit the model to the resampled data
        res_c = minimize(lambda p: ((optimal_model(x_resample, p[0], p[1]) - y_resample)**2).sum(), best_fit_param, method='Nelder-Mead')
        c0_bs_c, k_bs_c = res_c.x
        # Calculate y-values for the model curve for this iteration
        y_model_bs[i, :] = optimal_model(x_model, c0_bs_c, k_bs_c)

    # Calculate lower and upper bounds of the confidence interval for each point on the x-axis
    alpha = 0.05
    lower_bound = np.percentile(y_model_bs, alpha/2*100, axis=0)
    upper_bound = np.percentile(y_model_bs, (1-alpha/2)*100, axis=0)

    # Calculate lower and upper bounds of the prediction interval for each point on the x-axis
    alpha = 0.05
    sigma_squared = ((y_data - optimal_model(x_data, best_fit_param[0], best_fit_param[1]))**2).sum()/(len(x_data) - len(best_fit_param))
    tval = t.ppf(1 - alpha/2, len(x_data) - len(best_fit_param))

    lower_bound_p = y_model_bs.mean(axis=0) - tval *np.sqrt(sigma_squared)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())
    upper_bound_p = y_model_bs.mean(axis=0) + tval *np.sqrt(sigma_squared)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())

    # Plot the model curve and the confidence interval bounds
    plt.scatter(x_data,y_data, label='data', marker='o', color='blue', s=7)
    plt.plot(x_model, y_optimal_model, 'r-', label='model')
    plt.plot(x_model, lower_bound, 'g--', label='95% CI')
    plt.plot(x_model, upper_bound, 'g--')
    plt.plot(x_model, lower_bound_p, 'm--', label='95% PI')
    plt.plot(x_model, upper_bound_p, 'm--')
    plt.title(f'{product_name}, optimal model {model_list[min_index]}', fontsize=15)
    plt.axhline(y = spec, color = 'g', linestyle = '-')
    x_axis=np.arange(0, x_model[-1]+1,4)
    plt.xticks(x_axis)
    plt.legend()
    plt.show()
    display(Math(latex_strings[min_index]))
    
else:
    # Store the y-values for the model curve for each bootstrap iteration for confidence and prediction intervals
    y_model_bs = np.zeros((n_iter, n_points))
    y_pred_bs = np.zeros((n_iter, n_points))

    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample = x_data[idx]
        y_resample = y_data[idx]
        # Fit the model to the resampled data
        res = minimize(lambda p: ((optimal_model(x_resample, p[0], p[1], p[2]) - y_resample)**2).sum(), best_fit_param, method='Nelder-Mead')
        c0_bs, k1_bs, k2_bs = res.x
        # Calculate y-values for the model curve for this iteration
        y_model_bs[i, :] = optimal_model(x_model, c0_bs, k1_bs, k2_bs)

    # Calculate lower and upper bounds of the confidence interval for each point on the x-axis
    alpha = 0.05
    lower_bound = np.percentile(y_model_bs, alpha/2*100, axis=0)
    upper_bound = np.percentile(y_model_bs, (1-alpha/2)*100, axis=0)

    # Calculate lower and upper bounds of the prediction interval for each point on the x-axis
    alpha = 0.05
    sigma_squared = ((y_data - optimal_model(x_data,best_fit_param[0], best_fit_param[1], best_fit_param[2]))**2).sum()/(len(x_data) - len(best_fit_param))
    tval = t.ppf(1 - alpha/2, len(x_data) - len(best_fit_param))

    lower_bound_p = y_model_bs.mean(axis=0) - tval *np.sqrt(sigma_squared)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())
    upper_bound_p = y_model_bs.mean(axis=0) + tval *np.sqrt(sigma_squared)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())

    # Plot the model curve and the confidence interval bounds
    plt.scatter(x_data,y_data, label='data', marker='o', color='blue', s=7)
    plt.plot(x_model, y_optimal_model, 'r-', label='model')
    plt.plot(x_model, lower_bound, 'g--', label='95% CI')
    plt.plot(x_model, upper_bound, 'g--')
    plt.plot(x_model, lower_bound_p, 'm--', label='95% PI')
    plt.plot(x_model, upper_bound_p, 'm--')
    plt.title(f'{product_name}, optimal model {model_list[min_index]}', fontsize=15)
    plt.axhline(y = spec, color = 'g', linestyle = '-')
    x_axis=np.arange(0, x_model[-1]+1,4)
    plt.xticks(x_axis)
    plt.legend()
    plt.show()
    display(Math(latex_strings[min_index]))

Bootstrap method for the calculation of confidence and prediction curves for the fitted curve of the 2nd best model

In [ ]:
n_points = 100

if min_index_2nd < 7:
    # Store the y-values for the model curve for each bootstrap iteration for confidence and prediction intervals
    y_model_bs_2nd = np.zeros((n_iter, n_points))
    y_pred_bs_2nd = np.zeros((n_iter, n_points))

    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample_2nd = x_data[idx]
        y_resample_2nd = y_data[idx]
        # Fit the model to the resampled data
        res_c_2nd = minimize(lambda p: ((optimal_model_2nd(x_resample_2nd, p[0], p[1]) - y_resample_2nd)**2).sum(), best_fit_param_2nd, method='Nelder-Mead')
        c0_bs2_c, k_bs2_c = res_c_2nd.x
        # Calculate y-values for the model curve for this iteration
        y_model_bs_2nd[i, :] = optimal_model_2nd(x_model, c0_bs2_c, k_bs2_c)

    # Calculate lower and upper bounds of the confidence interval for each point on the x-axis
    alpha = 0.05
    lower_bound_2nd = np.percentile(y_model_bs_2nd, alpha/2*100, axis=0)
    upper_bound_2nd = np.percentile(y_model_bs_2nd, (1-alpha/2)*100, axis=0)

    # Calculate lower and upper bounds of the prediction interval for each point on the x-axis
    alpha = 0.05
    sigma_squared_2nd = ((y_data - optimal_model_2nd(x_data, best_fit_param_2nd[0], best_fit_param_2nd[1]))**2).sum()/(len(x_data) - len(best_fit_param_2nd))
    tval_2nd = t.ppf(1 - alpha/2, len(x_data) - len(best_fit_param_2nd))

    lower_bound_p_2nd = y_model_bs_2nd.mean(axis=0) - tval_2nd *np.sqrt(sigma_squared_2nd)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())
    upper_bound_p_2nd = y_model_bs_2nd.mean(axis=0) + tval_2nd *np.sqrt(sigma_squared_2nd)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())

   # Plot the model curve and the confidence interval bounds
    plt.scatter(x_data,y_data, label='data', marker='o', color='blue', s=7)
    plt.plot(x_model, y_optimal_model_2nd, 'r-', label='model')
    plt.plot(x_model, lower_bound_2nd, 'g--', label='95% CI')
    plt.plot(x_model, upper_bound_2nd, 'g--')
    plt.plot(x_model, lower_bound_p_2nd, 'm--', label='95% PI')
    plt.plot(x_model, upper_bound_p_2nd, 'm--')
    plt.title(f'{product_name}, 2nd model {model_list[min_index_2nd]}', fontsize=15)
    plt.axhline(y = spec, color = 'g', linestyle = '-')
    plt.legend()
    plt.show()
    display(Math(latex_strings[min_index_2nd]))
    
else:
    # Store the y-values for the model curve for each bootstrap iteration for confidence and prediction intervals
    y_model_bs_2nd = np.zeros((n_iter, n_points))
    y_pred_bs_2nd = np.zeros((n_iter, n_points))

    # Perform bootstrap iterations
    for i in range(n_iter):
        # Resample the data with replacement
        idx = np.random.randint(0, len(x_data), len(x_data))
        x_resample_2nd = x_data[idx]
        y_resample_2nd = y_data[idx]
        # Fit the model to the resampled data
        res_2nd = minimize(lambda p: ((optimal_model_2nd(x_resample_2nd, p[0], p[1], p[2]) - y_resample_2nd)**2).sum(), best_fit_param_2nd, method='Nelder-Mead')
        c0_bs2, k1_bs2, k2_bs2 = res_2nd.x
        # Calculate y-values for the model curve for this iteration
        y_model_bs_2nd[i, :] = optimal_model_2nd(x_model, c0_bs2, k1_bs2, k2_bs2)

    # Calculate lower and upper bounds of the confidence interval for each point on the x-axis
    alpha = 0.05
    lower_bound_2nd = np.percentile(y_model_bs_2nd, alpha/2*100, axis=0)
    upper_bound_2nd = np.percentile(y_model_bs_2nd, (1-alpha/2)*100, axis=0)

    # Calculate lower and upper bounds of the prediction interval for each point on the x-axis
    alpha = 0.05
    sigma_squared_2nd = ((y_data - optimal_model_2nd(x_data, best_fit_param_2nd[0], best_fit_param_2nd[1], best_fit_param_2nd[2]))**2).sum()/(len(x_data) - len(best_fit_param_2nd))
    tval_2nd = t.ppf(1 - alpha/2, len(x_data) - len(best_fit_param_2nd))

    lower_bound_p_2nd = y_model_bs_2nd.mean(axis=0) - tval_2nd *np.sqrt(sigma_squared_2nd)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())
    upper_bound_p_2nd = y_model_bs_2nd.mean(axis=0) + tval_2nd *np.sqrt(sigma_squared_2nd)* np.sqrt(1 + 1/len(x_data) + (x_model - x_data.mean())**2/((x_data - x_data.mean())**2).sum())
    
    # Plot the model curve and the confidence interval bounds
    plt.scatter(x_data,y_data, label='data', marker='o', color='blue', s=7)
    plt.plot(x_model, y_optimal_model_2nd, 'r-', label='model')
    plt.plot(x_model, lower_bound_2nd, 'g--', label='95% CI')
    plt.plot(x_model, upper_bound_2nd, 'g--')
    plt.plot(x_model, lower_bound_p_2nd, 'm--', label='95% PI')
    plt.plot(x_model, upper_bound_p_2nd, 'm--')
    plt.title(f'{product_name}, 2nd model {model_list[min_index_2nd]}', fontsize=15)
    plt.axhline(y = spec, color = 'g', linestyle = '-')
    x_axis=np.arange(0, x_model[-1]+1,4)
    plt.xticks(x_axis)
    plt.legend()
    plt.show()
    display(Math(latex_strings[min_index_2nd]))

In [ ]:
# Create 'results' directory if it doesn't exist
if not os.path.exists('results'):
    os.makedirs('results')   

print("Best model,",model_list[min_index])
if min_index < 7:
    parameters_df=pd.DataFrame({'Product/parameter': product_name, 'model': model_list[min_index],
                               '[Rel. Sub.]0':best_fit_param[0], 'k':best_fit_param[1],
                               'Confidence intervals for c0':c0_ci, 'Confidence intervals for k':k_ci})
    print(f"[Rel. Sub.]0 = {best_fit_param[0]:.2f}")
    print(f"k = {best_fit_param[1]:.4f}")
    print(f"Confidence intervals for c0: {c0_ci[0]:.4f} to {c0_ci[1]:.4f}")
    print(f"Confidence intervals for k: {k_ci[0]:.4f} to {k_ci[1]:.4f}")
    print("\n")
else:
    parameters_df=pd.DataFrame({'Product/parameter': product_name, 'model': model_list[min_index],
                               '[Rel. Sub.]0':best_fit_param[0], 'k':best_fit_param[1],
                               'Confidence intervals for c0':c0_ci, 'Confidence intervals for k1':k1_ci,
                               'Confidence intervals for k2':k2_ci})
    print(f"[Rel. Sub.]0 = {best_fit_param[0]:.2f}")
    print(f"k1 = {best_fit_param[1]:.4f}")
    print(f"k2 = {best_fit_param[2]:.4f}")
    print(f"Confidence intervals for c0: {c0_ci[0]:.4f} to {c0_ci[1]:.4f}")
    print(f"Confidence intervals for k1: {k1_ci[0]:.4f} to {k1_ci[1]:.4f}")
    print(f"Confidence intervals for k2: {k2_ci[0]:.4f} to {k2_ci[1]:.4f}")
    print("\n")
    
result_df = pd.DataFrame({'time': x_model, product_name: y_optimal_model, '95% LCI': lower_bound, 
                          '95% UCI': upper_bound, '95% LPI':lower_bound_p, '95% UPI':upper_bound_p})
result_df.to_csv(os.path.join('results', f'RelSub_calculation1st_{product_name}.csv'), index=False)
result_df.to_excel(os.path.join('results', f'RelSub_calculation1st_{product_name}.xlsx'), index=False)
parameters_df.to_excel(os.path.join('results', f'RelSub_parameters1st_{product_name}.xlsx'), index=False)

print("2nd Best model,",model_list[min_index_2nd])
if min_index_2nd < 7:
    parameters_df=pd.DataFrame({'Product/parameter': product_name, 'model': model_list[min_index_2nd],
                               '[Rel. Sub.]0':best_fit_param_2nd[0], 'k':best_fit_param_2nd[1],
                               'Confidence intervals for c0':c0_ci_2nd, 'Confidence intervals for k':k_ci_2nd})
    print(f"[Rel. Sub.]0 = {best_fit_param_2nd[0]:.2f}")
    print(f"k = {best_fit_param_2nd[1]:.4f}")
    print(f"Confidence intervals for c0: {c0_ci_2nd[0]:.4f} to {c0_ci_2nd[1]:.4f}")
    print(f"Confidence intervals for k: {k_ci_2nd[0]:.4f} to {k_ci_2nd[1]:.4f}")
    
else:
    parameters_df=pd.DataFrame({'Product/parameter': product_name, 'model': model_list[min_index_2nd],
                               '[Rel. Sub.]0':best_fit_param_2nd[0], 'k1':best_fit_param_2nd[1], 'k2':best_fit_param_2nd[2],
                               'Confidence intervals for c0':c0_ci, 'Confidence intervals for k1':k1_ci,
                               'Confidence intervals for k2':k2_ci})
    print(f"[Rel. Sub.]0 = {best_fit_param_2nd[0]:.2f}")
    print(f"k1 = {best_fit_param_2nd[1]:.4f}")
    print(f"k2 = {best_fit_param_2nd[2]:.4f}")
    print(f"Confidence intervals for c0: {c0_ci_2nd[0]:.4f} to {c0_ci_2nd[1]:.4f}")
    print(f"Confidence intervals for k1: {k1_ci_2nd[0]:.4f} to {k1_ci_2nd[1]:.4f}")
    print(f"Confidence intervals for k2: {k2_ci_2nd[0]:.4f} to {k2_ci_2nd[1]:.4f}")
    
result_df = pd.DataFrame({'time': x_model, product_name: y_optimal_model_2nd, '95% LCI': lower_bound_2nd, 
                          '95% UCI': upper_bound_2nd, '95% LPI':lower_bound_p_2nd, '95% UPI':upper_bound_p_2nd})
result_df.to_csv(os.path.join('results', f'RelSub_calculation2nd_{product_name}.csv'), index=False)
result_df.to_excel(os.path.join('results', f'RelSub_calculation2nd_{product_name}.xlsx'), index=False)
parameters_df.to_excel(os.path.join('results', f'RelSub_parameters2nd_{product_name}.xlsx'), index=False)

Making a prediction for a longer time period for the same fitted parameters as the optimal model

In [ ]:
predic_period = int(input("Months of prediction = "))

In [ ]:
x_model_p = np.linspace(min(x_data), predic_period, num=200)
y_model_p = optimal_model(x_model_p, *best_fit_param)

x_model_p_2nd = np.linspace(min(x_data), predic_period, num=200)
y_model_p_2nd = optimal_model_2nd(x_model_p_2nd, *best_fit_param_2nd)

In [ ]:
plt.title('Prediction for best model '+ model_list[min_index], fontsize=15)
plt.scatter(x_data,y_data, label="Experimental data")
plt.xlabel("months (-)")
plt.ylabel("Rel. Sub. (%)")
plt.plot(x_model_p,y_model_p, color='orange', label="model data")

plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.xlim([0, x_model_p[-1]])
x_axis=np.arange (0,x_model_p[-1]+1,4)
y_axis=np.arange(np.round(best_fit_param[0]-0.1),spec+0.1,0.05)
plt.xticks(x_axis)
plt.yticks(y_axis)
plt.legend()
plt.show()
display(Math(latex_strings[min_index]))

print(f"The value for the impurity at {x_model_p[-1]:.0f} months will be {y_model_p[-1]:.2f}%")

In [ ]:
plt.title('Prediction for 2nd best model '+ model_list[min_index_2nd], fontsize=15)
plt.scatter(x_data,y_data, label="Experimental data")
plt.xlabel("months (-)")
plt.ylabel("Rel. Sub. (%)")
plt.plot(x_model_p_2nd,y_model_p_2nd, color='orange', label="model data")

plt.axhline(y = spec, color = 'g', linestyle = '-')
plt.xlim([0, x_model_p_2nd[-1]])
x_axis=np.arange (0,x_model_p_2nd[-1]+1,4)
y_axis=np.arange(np.round(best_fit_param_2nd[0]-0.1),spec+0.1,0.05)
plt.xticks(x_axis)
plt.yticks(y_axis)
plt.legend()
plt.show()
display(Math(latex_strings[min_index_2nd]))

print(f"The value for the impurity at {x_model_p_2nd[-1]:.0f} months will be {y_model_p_2nd[-1]:.2f}%")

In [ ]:
from scipy.optimize import fsolve

def find_intersection(func, target_y, x_range, *params):
    # Define a function that represents the difference between the model and the target y
    diff_func = lambda x: func(x, *params) - target_y
    
    # Use fsolve to find the root (intersection point)
    intersection_x = fsolve(diff_func, x_range)

    return intersection_x[0] if len(intersection_x) > 0 else None

# Find the intersection for spec limit
target_y = spec
x_range = (min(x_model_p), max(x_model_p))  # Adjust the range based on your data
intersection_x = find_intersection(optimal_model, target_y, x_range, *best_fit_param)

if intersection_x is not None:
    print(f'The time value where the optimal model intersects with {target_y}% is at {intersection_x:.2f} months')
else:
    print(f'No intersection found within the given range.')

In [ ]:
# Find the intersection for spec limit
target_y = spec
x_range_2nd = (min(x_model_p), max(x_model_p))  # Adjust the range based on your data
intersection_x_2nd = find_intersection(optimal_model_2nd, target_y, x_range_2nd, *best_fit_param_2nd)

if intersection_x_2nd is not None:
    print(f'The time value where the 2nd optimal model intersects with {target_y}% is at {intersection_x_2nd:.2f} months')
else:
    print(f'No intersection found within the given range.')